Webscraping neighbourhood income data

In [ ]:
import requests
import pandas as pd
from bs4 import BeautifulSoup

# URL of the webpage
url = "https://www03.cmhc-schl.gc.ca/hmip-pimh/en/TableMapChart/TableMatchingCriteria?GeographyType=MetropolitanMajorArea&GeographyId=2270&CategoryLevel1=Population%2C%20Households%20and%20Housing%20Stock&CategoryLevel2=Household%20Income&ColumnField=HouseholdIncomeRange&RowField=Neighbourhood&SearchTags%5B0%5D.Key=Households&SearchTags%5B0%5D.Value=Number&SearchTags%5B1%5D.Key=Statistics&SearchTags%5B1%5D.Value=AverageAndMedian"

# Send a GET request to the URL
response = requests.get(url)
html_content = response.content

# Parse the HTML content using BeautifulSoup
soup = BeautifulSoup(html_content, 'html.parser')

# Find the table containing the neighborhood and median income data
table = soup.find('table', class_='CawdDataTable')

# List of neighborhoods to exclude
excluded_neighborhoods = {
    'Ajax', 'Aurora', 'Bradford', 'Brampton (East)', 'Brampton (West)', 
    'Caledon', 'East Gwillimbury', 'Georgina', 'Halton Hills', 'Milton', 
    'Mississauga Centre', 'Mono', 'New Tecumseth', 'Newmarket', 
    'Oakville (excl. Bronte)', 'Orangeville', 'Pickering', 
    'Richmond Hill', 'Stouffville', 'Uxbridge', 'Vaughan', 'Whitchurch', "Bronte",
    "Churchill Meadows", "City Centre North", "City Centre South", "Clarkson",
    "Cooksville", "Crescent Town", "Davenport", "Erin Mills",
    "King", "King West", "Lorne Park", "Malton", "Markham", "Meadowvale",
    "Moore Park", "Port Credit", "Streetsville", "West Gwillimbury"
}

# List of neighbourhood replacements
replacements = {
    "Bendale": ["Bendale South", "Bendale-Glen Andrew"],
    "Agincourt": ["Agincourt North", "Agincourt South-Malvern West"],
    "Willowdale East": ["Avondale", "Willowdale East", "Yonge-Doris"],
    "Bay Street Corridor": ["Bay-Cloverhill", "Yonge-Bay Corridor"],
    "Church-Yonge Corridor": ["Church-Wellesley", "Downtown Yonge East"],
    "Dovercourt": ["Dovercourt Village", "Junction-Wallace Emerson"],
    "Downsview": ["Downsview", "Oakdale-Beverly Heights"],
    "Parkwoods-Donalda": ["Fenside-Parkwoods", "Parkwoods-O'Connor Hills"],
    "Woburn": ["Golfdale-Cedarbrae-Woburn", "Woburn North"],
    "Waterfront Communities -The Island": ["Harbourfront-CityPlace", "St Lawrence-East Bayfront The Islands", "Wellington Place"],
    "Mimico": ["Humber Bay Shores", "Mimico-Queensway"],
    "L'Amoreaux": ["East L'Amoreaux", "West L'Amoreaux"],
    "Malvern": ["Malvern East", "Malvern West"],
    "Rouge": ["Morningside", "Rouge"],
    "Morningside": ["Morningside Heights"],
    "Riverdale": ["North Riverdale", "South Riverdale"],
    "Mount Pleasant West": ["North Toronto","South Eglinton-Davisville"],
    "Weston": ["Weston", "Weston-Pelham Park"]
}

# Initialize lists to store neighborhood names and median incomes
neighborhoods = []
median_incomes = []

# Loop through each row of the table, except the header and footer
for row in table.find_all('tr')[1:-1]:  # skip header and footer rows
    cells = row.find_all('td')
    if len(cells) > 1:
        neighborhood = row.find('th').get_text(strip=True)
        median_income = cells[3].get_text(strip=True).replace(',', '').replace('$', '')  # Clean up the income value
        median_income = int(median_income)  # Convert to integer

        # Split neighborhoods with a slash and duplicate the median income
        for nb in neighborhood.split('/'):
            nb = nb.strip()  # Remove leading and trailing whitespace
            if nb in replacements:  # Check if the neighborhood has replacements
                for new_nb in replacements[nb]:
                    if new_nb not in excluded_neighborhoods:  # Check if new neighborhood is excluded
                        neighborhoods.append(new_nb)  # Add the new neighborhood
                        median_incomes.append(median_income)  # Add the same median income
            elif nb not in excluded_neighborhoods:  # If no replacements and not excluded
                neighborhoods.append(nb)  # Add the split neighborhood
                median_incomes.append(median_income)  # Add the same median income

# Create a DataFrame from the lists
income_df = pd.DataFrame({
    'Neighbourhood': neighborhoods,
    'Median Income Before Tax': median_incomes
})

# Write the DataFrame to a CSV file
csv_file_path = 'median_income_by_neighbourhood.csv'
income_df.to_csv(csv_file_path, index=False)

# Display the DataFrame
print(income_df)
print(f'Data has been written to {csv_file_path}')


                    Neighbourhood  Median Income Before Tax
0                         Toronto                     97000
1                 Agincourt North                     89000
2    Agincourt South-Malvern West                     89000
3                    Malvern East                     89000
4                    Malvern West                     89000
..                            ...                       ...
141             Greenwood-Coxwell                     95000
142              Woodbine-Lumsden                     96000
143                      Wychwood                     86000
144                Yonge-Eglinton                     94000
145               Yonge-St. Clair                     94000

[146 rows x 2 columns]
Data has been written to median_income_by_neighbourhood.csv


Finding the water levels at black creek and don river for flooding event of interest (july 8th 2013)

In [5]:
import pandas as pd

# Load the data file
file_name = 'flow data.csv'
data = pd.read_csv(file_name, header=None, skiprows=1)

# Assign column names
data.columns = ['ID', 'PARAM', 'Date', 'Value', 'SYM']

# Convert 'Date' to datetime format and filter 'PARAM' column for valid values
data['Date'] = pd.to_datetime(data['Date'], errors='coerce')
data = data[data['PARAM'].isin(['1', '2'])].copy()
data['PARAM'] = data['PARAM'].astype(int)

# Filter for July 8, 2013, and parameter 2 (level data)
july_8_2013_level_data = data[(data['PARAM'] == 2) & (data['Date'] == '2013-07-08')]

july_8_2013_level_data = july_8_2013_level_data [['ID','Date','Value']]

# Display the filtered data
print(july_8_2013_level_data)


            ID       Date   Value
25207  02HC027 2013-07-08   1.426
55249  02HC024 2013-07-08  13.026


Finding the baseline water levels for these rivers

In [ ]:
# Filter only for PARAM = 2 (water level data) and convert 'Value' to numeric
water_level_data = data[data['PARAM'] == 2]
water_level_data['Value'] = pd.to_numeric(water_level_data['Value'], errors='coerce')

# Filter data for July across all years
july_water_level_data = water_level_data[water_level_data['Date'].dt.month == 7]

# Calculate the average (baseline) water level for each gauge for July
baseline_july_water_levels = july_water_level_data.groupby('ID')['Value'].mean().reset_index()
baseline_july_water_levels.columns = ['ID', 'Baseline_July_Water_Level']

# Display the result
print(baseline_july_water_levels)


july basline water level         ID  Baseline_July_Water_Level
0  02HC024                  12.195959
1  02HC027                   0.381672


C:\Users\mayac\AppData\Local\Temp\ipykernel_14700\2444658108.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  water_level_data['Value'] = pd.to_numeric(water_level_data['Value'], errors='coerce')


Extracting the climate data

In [9]:
# Load the data file
climate_file = 'climate-daily.csv'
climate_data = pd.read_csv(climate_file)

# Convert 'LOCAL_DATE' to datetime format
climate_data['LOCAL_DATE'] = pd.to_datetime(climate_data['LOCAL_DATE'], errors='coerce')

# Filter for data from July 7th and 8th, 2013
july_7_8_data = climate_data[(climate_data['LOCAL_DATE'] == '2013-07-07') | (climate_data['LOCAL_DATE'] == '2013-07-08')]

# Select only the columns of interest: 'x', 'y', 'STATION_NAME', 'LOCAL_DATE', and 'TOTAL_PRECIPITATION'
july_7_8_selected_columns = july_7_8_data[['x', 'y', 'STATION_NAME', 'LOCAL_DATE', 'TOTAL_PRECIPITATION']]

# Display the filtered data
print(july_7_8_selected_columns)


      x          y  STATION_NAME LOCAL_DATE  TOTAL_PRECIPITATION
6 -79.4  43.666667  TORONTO CITY 2013-07-07                 38.5
7 -79.4  43.666667  TORONTO CITY 2013-07-08                 96.8
